In [ ]:
# preparations

from pathlib import Path
from dotenv import load_dotenv

load_dotenv(Path('..') / '.doc.env')

import os
user_email = os.environ['USERNAME']
password = os.environ['PASSWORD']

BASE_URL = 'https://vault.bitwarden.com/api'
IDENTITY_URL = 'https://vault.bitwarden.com/identity'


### First step requires us to get the number of iterations and the type for the key derivation algorithm

In [ ]:
import requests

prelogin_url = f'{BASE_URL}/accounts/prelogin'

response = requests.post(prelogin_url, json={
    'email': user_email
})
json_response = response.json()

kdf_type, iterations = json_response['Kdf'], json_response['KdfIterations']

print(iterations)

### The next step requires hashing the email and password

In [ ]:
import hashlib

def get_key(user_email, password, iterations):
    # apply KDF for N interations over 'password' using 'email' as salt
    e_password = hashlib.pbkdf2_hmac(
        hash_name='sha256',
        password=password.encode(),
        salt=user_email.lower().encode(),
        iterations=iterations,
        dklen=256/8)
    
    # apply KDF for 1 iteration over 'e_password' using 'password' as salt
    hash_password = hashlib.pbkdf2_hmac(
        hash_name='sha256',
        password=e_password,
        salt=password.encode(),
        iterations=1,
        dklen=256/8
    )

    return e_password, hash_password

enc_password, hash_password = get_key(user_email, password, iterations)

### Next is the actual login request

In [ ]:
# replace this with new 2fa code:
token_code = ''

In [ ]:
import base64

# need to base64 encode it so it can be transmitted in a json
hash_password = base64.b64encode(hash_password)

payload = {
    'grant_type': 'password',
    'username': user_email,
    'password': hash_password.decode('utf-8'),
    'scope': 'api offline_access',
    'client_id': 'web',
    'deviceType': 10,
    'deviceIdentifier': '403374ad-ab7b-441c-a74d-976412e10d3c',
    'deviceName': 'firefox',
    'twoFactorToken': token_code,
    'twoFactorProvider': 0,  # 1 for email - 5 for .. dunno
    'twoFactorRemember': 1  # set to 1 to remember two factor
}

login_response = requests.post(f'{IDENTITY_URL}/connect/token', data=payload)

print(login_response.status_code)

In [ ]:
from dataclasses import dataclass

@dataclass
class CipherString:
    enc_type: int
    iv: bytes
    data: bytes
    mac: bytes

    @classmethod
    def from_string(cls, s):
        enc_type, remain = s.split('.')
        if enc_type != '2':  # AesCbc256_HmacSha256_B64
            # note: we only have AesCbc256_HmacSha256_B64 for now
            raise Exception('Enc type not implemented.')

        iv, data, mac = [base64.b64decode(v) for v in remain.split('|')]

        return cls(enc_type=enc_type, iv=iv, mac=mac, data=data)

key_enc_string = CipherString.from_string(login_response.json()['Key'])

### Next is the master key expansion

In [ ]:
import struct
import hmac

# taken and adapted from: https://github.com/Legrandin/pycryptodome/blob/28c3af46ad7bd525f981c1f7e2b68d744bb953f5/lib/Crypto/Protocol/KDF.py#L275-L333
# HKDF consists of two steps, extract and expand. Bitwarden uses expand only, after PBKDF2
def hkdf_expand(master, key_len, hashmod, context=None):
    # Step 2: expand
    t = [ b'' ]
    n = 1
    t_len = 0
    while t_len < key_len:  # does only one cycle at size 32
        h_mac = hmac.new(master, t[-1] + context + struct.pack('B', n), digestmod=hashmod)

        t.append(h_mac.digest())
        t_len += 256/8  # digest_size
        n += 1

    return b''.join(t)[:key_len]


def expand_key(key):

    enc_k = hkdf_expand(
        master=key,
        key_len=32,
        hashmod='sha256',
        context=b'enc')

    mac_k = hkdf_expand(
        master=key,
        key_len=32,
        hashmod='sha256',
        context=b'mac')

    return enc_k, mac_k

enc_key, mac_key = expand_key(enc_password)

### Optional step: check mac on the master key to determine if it can be decrypted correctly

In [ ]:
def check_macs(key_mac, c_string):
    comp_mac = hmac.digest(key_mac, c_string.iv + c_string.data, 'sha256')

    hmac1 = hmac.digest(key_mac, c_string.mac, 'sha256')
    hmac2 = hmac.digest(key_mac, comp_mac, 'sha256')

    return hmac1 == hmac2

print(check_macs(mac_key, key_enc_string))

### Decrypt the master key using the user key

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad


def decrypt_aes(key, iv, data):
    cipher = AES.new(key, AES.MODE_CBC, IV=iv)
    plain = cipher.decrypt(data)
    return unpad(plain, AES.block_size)

p_key = decrypt_aes(enc_key, key_enc_string.iv, key_enc_string.data)
p_key_enc = p_key[:32]
p_key_mac = p_key[32:64]

### Check mac is valid for the encrypted string

In [ ]:
enc_string = 'ENTER YOUR ENCRYPTED STRING HERE'
enc_string = CipherString.from_string(enc_string)

print(check_macs(p_key_mac, enc_string))

### Finally, decrypt the string itself

In [ ]:
decrypt_aes(p_key_enc, enc_string.iv, enc_string.data)